## Titanic Disaster Dataset Feature Engineering

This notebook is an attempt towards kick-starting my way around the Titanic dataset to improve my model scores based on feature engineering techniques.
This is not going to be an extensive EDA notebook for viewers, as there are already tons of notebooks available with great amount of EDA to understand the intricacies of the dataset from all aspects.
* We will work with missing value imputations
* We will also try to combine some features with each other to come up with new ones
* We will try to improve our scores by building up on our feature engineering process

In [2]:
# Import the required packages here

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore') #Remove warning messages within the kernel

In [3]:
# Import the datasets here
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [5]:
# Append the train and test dataframes for data cleaning. 
# We also add a flag so that at a later stage, we can split the combined dataframe to train and test again

train['test_flag'] = 0
test['test_flag'] = 1
df_combined = pd.concat([train, test], axis=0, copy=True)

### High level EDA

We take a look at the % of missing values in each of the features available in the train and test dataset.

_Ignore the missing values in the 'Survived' feature, because of the concatenation the missing values are attributed from the test set_

In [6]:
# Check the % missing values in all the columns of the train set
print(df_combined.isnull().sum()*100/df_combined.shape[0])

# Ignore the missing values for the output class 'Survived' as they are from the test set

PassengerId     0.000000
Survived       31.932773
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.091673
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.076394
Cabin          77.463713
Embarked        0.152788
test_flag       0.000000
dtype: float64


In [7]:
# Subsetting for the list of columns which has less to no missing values

df_subset = df_combined[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Name', 'Embarked', 'test_flag']]

### Data Cleaning and missing value imputation for the columns

* One of the most common approach to feature engineering has been using the **Name** feature to create a Title feature using regex
* Using the **Cabin** feature to create classes from the initial letter of the column. The nan have been tagged as 'U' or 'Unknown' class in the data
* We'll not use the **Ticket** feature for now, as that feature didn't make quite a lot of sense to me


In [8]:
# Cleaning and level modifications for the categorical features

for dataset in df_subset:
    df_subset['Title'] = df_subset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df_subset['Title'], df_subset['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,4
Countess,1,0
Don,0,1
Dona,1,0
Dr,1,7
Jonkheer,0,1
Lady,1,0
Major,0,2


In [9]:
for dataset in df_subset:
    df_subset['Title'] = df_subset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
     'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    df_subset['Title'] = df_subset['Title'].replace('Mlle', 'Miss')
    df_subset['Title'] = df_subset['Title'].replace('Ms', 'Miss')
    df_subset['Title'] = df_subset['Title'].replace('Mme', 'Mrs')
    
df_subset[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [10]:
df_subset['Cabin'] = df_subset['Cabin'].replace(np.nan, 'U')
df_subset['Cabin_Class'] = df_subset['Cabin'].astype(str).str[0]

In [11]:
# Remove the columns here, for which we have generated our own classes

df_subset = df_subset.drop(['Ticket', 'Cabin', 'Name'], axis = 1)

### Age

* We find the median of the **Age** feature to impute the missing ages, which is roughly about 20%
* We use the same **Age** column to create buckets of age groups. For _less than 18 (1)_ and for _greater than or equal to 18 (0)_

In [12]:
median_age = df_subset["Age"].median()
df_subset["Age"].fillna(median_age, inplace=True)

In [13]:
df_subset.Age[df_subset.Age < 18] = 1
df_subset.Age[df_subset.Age >= 18] = 0

In [14]:
df_subset['Age'] = df_subset['Age'].astype(int) # Convert the age column to int type to be able to perform operations

### SibSp & Parch

* We use the **Siblings** and the **Parents/Children** features combined to generate a feature called **Family Size**
* Having created a new feature out of the above two, we drop the original features
* We also use the **Family Size** feature to create buckets. _Family Size = 1 (0), Family Size greater than 1 and less than or equal to 4 (1), Family Size greater than 4 (2)_

In [15]:
for dataset in df_subset:
    df_subset['FamilySize'] = df_subset['SibSp'] + df_subset['Parch'] + 1

In [16]:
df_subset = df_subset.drop(['SibSp', 'Parch'], axis = 1)

In [17]:
df_subset.FamilySize[df_subset.FamilySize == 1] = 0
df_subset.FamilySize[(df_subset.FamilySize > 1) & (df_subset.FamilySize <= 4)] = 1
df_subset.FamilySize[(df_subset.FamilySize > 4)] = 2

### Fare

* The **Fare** column has been used to create buckets of Fare class
* Firstly, the missing values (0.07%) are replaced using Median imputation
* The buckets are Fare <= 7.91 (0), Fare > 7.91 and Fare <= 14.454 (1), Fare > 14.454 and Fare <= 31 (2), Fare > 31 (3)
* The Fare buckets have been created based on the percentile of data availabilities

In [18]:
df_subset['Fare'].describe()

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [19]:
df_subset['Fare'].fillna(df_subset['Fare'].dropna().median(), inplace=True)

In [20]:
df_subset.Fare[df_subset.Fare <= 7.91] = 0
df_subset.Fare[(df_subset.Fare > 7.91) & (df_subset.Fare <= 14.454)] = 1
df_subset.Fare[(df_subset.Fare > 14.454) & (df_subset.Fare <= 31)] = 2
df_subset.Fare[df_subset.Fare > 31] = 3
df_subset['Fare'] = df_subset['Fare'].astype(int) # Convert the fare column to integer type

df_subset.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,test_flag,Title,Cabin_Class,FamilySize
0,1,0.0,3,male,0,0,S,0,Mr,U,1
1,2,1.0,1,female,0,3,C,0,Mrs,C,1
2,3,1.0,3,female,0,1,S,0,Miss,U,0
3,4,1.0,1,female,0,3,S,0,Mrs,C,1
4,5,0.0,3,male,0,1,S,0,Mr,U,0


### Embarked

* We do a simple mode imputation in the Embarked feature which attributes to approximately 0.15% missing

In [21]:
mode_embarked = df_subset.Embarked.dropna().mode()[0]
df_subset["Embarked"].fillna(mode_embarked, inplace=True)

In [22]:
# In this step, we use the factorize method to factorize the categorical features that we have created above.
# This is done for model interpretability

cols_new = ['Embarked', 'Title', 'Sex', 'Cabin_Class']

for col in cols_new:
    df_subset[col] = pd.factorize(df_subset[col])[0] + 1
    
df_subset.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,test_flag,Title,Cabin_Class,FamilySize
0,1,0.0,3,1,0,0,1,0,1,1,1
1,2,1.0,1,2,0,3,2,0,2,2,1
2,3,1.0,3,2,0,1,1,0,3,1,0
3,4,1.0,1,2,0,3,1,0,2,2,1
4,5,0.0,3,1,0,1,1,0,1,1,0


In [23]:
# We split the combined dataset to train and test set again

train_set = df_subset[df_subset['test_flag']==0]
test_set = df_subset[df_subset['test_flag']==1]

train_set['Survived'] = train_set['Survived'].astype(int) # Convert the target feature to integer

In [24]:
# Check if the train and test split was done properly

print(train_set.shape)
print(test_set.shape)

(891, 11)
(418, 11)


In [25]:
# Remove the columns which are not required. Test Flag is not required since we split the data already, and the Survived column is redundant in the test set

test_set = test_set.drop(['Survived', 'test_flag'], axis = 1)
train_set = train_set.drop(['test_flag'], axis = 1)

In [26]:
train_set.head(5)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Cabin_Class,FamilySize
0,1,0,3,1,0,0,1,1,1,1
1,2,1,1,2,0,3,2,2,2,1
2,3,1,3,2,0,1,1,3,1,0
3,4,1,1,2,0,3,1,2,2,1
4,5,0,3,1,0,1,1,1,1,0


### Model Implementation Process

We use the engineered features into the models to check out the scores. Below are the set of models that we use for this Kernel
1. Logistic Regression
2. Decision Tree
3. Random Forest
4. Naive Bayes

In [27]:
from sklearn.model_selection import train_test_split

X = train_set[['Age', 'Sex', 'Embarked', 'Title', 'Pclass', 'FamilySize']]
y = train_set[['Survived']]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(596, 6) (596, 1)
(295, 6) (295, 1)


In [29]:
X_test.head()

,Age,Sex,Embarked,Title,Pclass,FamilySize
709,0,1,2,4,3,1
439,0,1,1,1,2,0
840,0,1,1,1,3,0
720,1,2,1,3,2,1
39,1,2,2,3,3,1


In [30]:
# Logistic Regression Model

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logistic_model = logreg.fit(X_train, y_train)
predictions = logistic_model.predict(X_test)

acc_logistic_regressions = round(logistic_model.score(X_train, y_train) * 100, 2)
acc_logistic_regressions

80.2

In [31]:
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
decision_tree = dec_tree.fit(X_train, y_train)
prediction_tree = decision_tree.predict(X_test)

acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

85.57

In [32]:
# Random Forest Model

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
random_forest = rf.fit(X_train, y_train)
prediction_forest = random_forest.predict(X_test)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

85.57

In [33]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gaussian = gnb.fit(X_train, y_train)
prediction_naive = gaussian.predict(X_test)

acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
acc_gaussian

77.68

In [34]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=10,
#     verbose=5,
)

catboost = clf.fit(X_train, y_train,
    # cat_features=cat_features,
    eval_set=(X_test, y_test),
)

Learning rate set to 0.206726
0:	learn: 0.6288767	test: 0.6266781	best: 0.6266781 (0)	total: 51ms	remaining: 459ms
1:	learn: 0.5801920	test: 0.5814590	best: 0.5814590 (1)	total: 51.3ms	remaining: 205ms
2:	learn: 0.5408101	test: 0.5415290	best: 0.5415290 (2)	total: 51.6ms	remaining: 120ms
3:	learn: 0.5093224	test: 0.5076102	best: 0.5076102 (3)	total: 51.8ms	remaining: 77.7ms
4:	learn: 0.4914277	test: 0.4880850	best: 0.4880850 (4)	total: 52.1ms	remaining: 52.1ms
5:	learn: 0.4742430	test: 0.4719607	best: 0.4719607 (5)	total: 52.4ms	remaining: 34.9ms
6:	learn: 0.4615335	test: 0.4589765	best: 0.4589765 (6)	total: 52.6ms	remaining: 22.6ms
7:	learn: 0.4490427	test: 0.4493682	best: 0.4493682 (7)	total: 52.9ms	remaining: 13.2ms
8:	learn: 0.4403939	test: 0.4431770	best: 0.4431770 (8)	total: 53.5ms	remaining: 5.94ms
9:	learn: 0.4316692	test: 0.4372236	best: 0.4372236 (9)	total: 54.1ms	remaining: 0us

bestTest = 0.4372236135
bestIteration = 9



In [36]:
acc_catboost = round(catboost.score(X_train, y_train) * 100, 2)
acc_catboost

84.4

It is observed that the best accuracy is obtained in the Random Forest Model, which we will use to make our submissions to the competition.

### Use the section below to submit on the test set

In [37]:
test_feature = test_set[['Age', 'Sex', 'Embarked', 'Title', 'Pclass', 'FamilySize']]
test_id = test_set['PassengerId']
test_feature.head()

,Age,Sex,Embarked,Title,Pclass,FamilySize
0,0,1,3,1,3,0
1,0,2,1,2,3,1
2,0,1,3,1,2,0
3,0,1,1,1,3,0
4,0,2,1,2,3,1


In [38]:
test_predictions = catboost.predict(test_feature)

In [39]:
output = pd.DataFrame({'PassengerId' : test_id, 'Survived': test_predictions})
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [40]:
output.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# Use this data type conversion for the output feature, if the score comes as 0 in submission

# model.predict(test_data).astype(int)